In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import precision_recall_curve

from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import FunctionTransformer, StandardScaler

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_recall_fscore_support
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from skimage.transform import resize
from skimage.feature import canny

from scipy.ndimage import find_objects, label
import matplotlib as mpl


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from joblib import dump



In [ ]:
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
X = mnist["data"]
y = mnist["target"].astype(np.uint8)


In [ ]:
type(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) 
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0
X_val_normalized = X_val / 255.0

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)

plt.bar(unique, counts)
plt.title("Frequency of Each Digit in the Dataset")
plt.xlabel("Digit")
plt.ylabel("Frequency")
plt.xticks(unique)
plt.show()

In [ ]:
# Visualize pixel intensity distribution of a sample
plt.hist(X_train_normalized[0], bins=20, range=[0,1])
plt.title("Pixel Intensity Distribution for a Sample Digit")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.show()

# Example: Visualizing multiple samples
fig, axes = plt.subplots(10, 10, figsize=(9, 9),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, ax in enumerate(axes.flat):
    ax.imshow(X[i].reshape(28, 28), cmap='binary', interpolation='nearest')
    ax.text(0.05, 0.05, str(y[i]),
            transform=ax.transAxes, color='green')

In [ ]:
import numpy as np

unique_digits = np.unique(y_train)

for digit in unique_digits:
    digit_indices = np.where(y_train == digit)[0]
    digit_images = X_train[digit_indices]
    
    # Assuming the images are flattened. If not, you might need to adjust.
    max_intensity = np.max(digit_images)
    min_intensity = np.min(digit_images)
    avg_intensity = np.mean(digit_images)
    
    print(f"Digit: {digit}, Max intensity: {max_intensity}, Min intensity: {min_intensity}",f"Digit: {digit}, Average intensity: {avg_intensity}")
    print()





In [ ]:
import numpy as np
import matplotlib.pyplot as plt

unique_digits = np.unique(y_train)
avg_intensity = []

for digit in unique_digits:
    digit_indices = np.where(y_train == digit)[0]
    digit_images = X_train[digit_indices]
    
    # Calculate the average intensity for each digit
    avg_intensity.append(np.mean(digit_images))

# Plotting
plt.bar(unique_digits, avg_intensity, color='skyblue')
plt.xlabel('Digits')
plt.ylabel('Average Pixel Intensity')
plt.title('Average Pixel Intensity by Digit')
plt.xticks(unique_digits)
plt.show()


In [ ]:

knn_clf = KNeighborsClassifier(n_jobs= -1)
knn_clf.fit(X_train_normalized, y_train)



In [ ]:
# Evaluate on the validation set for hyperparameter tuning or model selection
val_accuracy = knn_clf.score(X_val_normalized, y_val)
print("Validation set accuracy:", val_accuracy)

# After finalizing the model and its hyperparameters, evaluate on the test set
test_accuracy = knn_clf.score(X_test_normalized, y_test)
print("Test set accuracy:", test_accuracy)

In [ ]:
dump(knn_clf, 'model_rf_clf.joblib')

In [ ]:


svm_clf = SVC(gamma='auto', kernel='rbf')
svm_clf.fit(X_train_normalized, y_train)


In [ ]:
# Evaluate on the validation set for hyperparameter tuning or model selection
val_accuracy_svm = svm_clf.score(X_val_normalized, y_val)
print("Validation set accuracy:", val_accuracy_svm)

# After finalizing the model and its hyperparameters, evaluate on the test set
test_accuracy_svm = svm_clf.score(X_test_normalized, y_test)
print("Test set accuracy:", test_accuracy_svm)

In [ ]:
dump(svm_clf, 'svm_clf.joblib')

In [ ]:

rf_clf = RandomForestClassifier(n_estimators=100)
rf_clf.fit(X_train_normalized, y_train)

In [ ]:
# Evaluate on the validation set for hyperparameter tuning or model selection
val_accuracy_clf = rf_clf.score(X_val_normalized, y_val)
print("Validation set accuracy:", val_accuracy_clf)

# After finalizing the model and its hyperparameters, evaluate on the test set
test_accuracy_clf = rf_clf.score(X_test_normalized, y_test)
print("Test set accuracy:", test_accuracy_clf)

In [ ]:

dump(rf_clf, 'model_rf_clf.joblib')


Gör allt på en gång

In [ ]:
# List of models and their parameter grids
models_and_parameters = [
    (SVC(), {
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto'],
        'kernel': ['rbf', 'linear']
    }),
    (RandomForestClassifier(), {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }),
    (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    })
]

# Initialize a dictionary to hold the best models
best_models = {}

# Loop over each model and its hyperparameters
for model, param_grid in models_and_parameters:
    print(f"Starting GridSearchCV for {model.__class__.__name__}...")
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_normalized, y_train)
    
    # Save the best model
    best_models[model.__class__.__name__] = grid_search.best_estimator_
    
    print(f"Best parameters for {model.__class__.__name__}: {grid_search.best_params_}")
    print(f"Best cross-validation accuracy for {model.__class__.__name__}: {grid_search.best_score_}\n")

# Assuming the models were SVC, RandomForestClassifier, and KNeighborsClassifier,
# and they are now saved in best_models
voting_clf = VotingClassifier(
    estimators=[
        ('svc', best_models['SVC']),
        ('rf', best_models['RandomForestClassifier']),
        ('knn', best_models['KNeighborsClassifier'])
    ],
    voting='soft'  # Change to 'soft' if all models support predict_proba
)

# Fit the voting classifier on the training data
voting_clf.fit(X_train_normalized, y_train)

# Evaluate the voting classifier on the test data
test_accuracy = voting_clf.score(X_test_normalized, y_test)
print("Test set accuracy with Voting Classifier:", test_accuracy)


In [ ]:
print(best_models.keys())

# Explicitly mentioning the directory if necessary, for example, the current directory
dump(best_models['RandomForestClassifier'], 'rf_best_model.joblib')
dump(best_models['SVC'], 'svc_best_model.joblib')

dump(best_models['KNeighborsClassifier'], 'knn_best_model.joblib')


In [ ]:
from joblib import load
# Load the models
svc_best_model = load('svc_best_model.joblib')
rf_best_model = load('rf_best_model.joblib')
knn_best_model = load('knn_best_model.joblib')

In [ ]:
import os

for filename in os.listdir('.'):
    if filename.endswith('.joblib'):
        print(filename)

In [ ]:
# Assuming X_test_normalized and y_test are available and properly preprocessed

# Make predictions with SVC
svc_predictions = svc_best_model.predict(X_test_normalized)
# Make predictions with RandomForest
rf_predictions = rf_best_model.predict(X_test_normalized)
# Make predictions with KNeighborsClassifier
knn_predictions = knn_best_model.predict(X_test_normalized)
# Evaluate SVC model
print("SVC Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test, svc_predictions):.4f}")
print(classification_report(y_test, svc_predictions))

# Evaluate RandomForest model
print("RandomForest Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test, rf_predictions):.4f}")
print(classification_report(y_test, rf_predictions))

# Evaluate KNeighborsClassifier model
print("KNeighborsClassifier Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test, knn_predictions):.4f}")
print(classification_report(y_test, knn_predictions))


SVC vann.. går idare och tränar med mer data

In [ ]:
from sklearn.ensemble import BaggingClassifier


# Instantiate SVC with the best parameters
svc_clf = SVC(C=10, gamma='scale', kernel='rbf', probability=True)

# Wrap it in a BaggingClassifier
bagging_clf = BaggingClassifier(svc_clf, n_estimators=10, max_samples=1.0, bootstrap=True, n_jobs=-1)

# Train the ensemble classifier on your training data
bagging_clf.fit(X_train_normalized, y_train)

# Evaluate the ensemble classifier on your test data
test_accuracy = bagging_clf.score(X_test_normalized, y_test)
print("Bagging SVC Test set accuracy:", test_accuracy)


In [ ]:
svc_clf = SVC(C=10, gamma='scale', kernel='rbf', probability=True)
svc_clf.fit(X_train_normalized, y_train)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs

# Create 40 separable points
X, y = make_blobs(n_samples=200, centers=2, random_state=6)

# Fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(X, y)

# Create grid to evaluate model
xx = np.linspace(min(X[:, 0]), max(X[:, 0]), 30)
yy = np.linspace(min(X[:, 1]), max(X[:, 1]), 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# Plot data points and color the areas of different classes
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, edgecolors='k')
plt.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
            linestyles=['--', '-', '--'])
# Plot support vectors
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
            linewidth=1, facecolors='none', edgecolors='k')
plt.title('SVM Decision Boundary with Support Vectors')
plt.show()


In [ ]:
# Print class information
print("Class labels:", np.unique(y))
print("Samples per class:", [np.sum(y == i) for i in np.unique(y)])

# Create grid to evaluate model
xx = np.linspace(min(X[:, 0]), max(X[:, 0]), 30)
yy = np.linspace(min(X[:, 1]), max(X[:, 1]), 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# Plot data points and color the areas of different classes
scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, edgecolors='k')
plt.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

# Plot support vectors
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
            linewidth=1, facecolors='none', edgecolors='k')
plt.title('SVM Decision Boundary with Support Vectors')
plt.legend(*scatter.legend_elements(), title="Classes")
plt.show()

In [ ]:
# Making predictions on the test set
predictions_bagging = bagging_clf.predict(X_test_normalized)
probabilities = bagging_clf.predict_proba(X_test_normalized)


In [ ]:
# Evaluate SVC model
print("SVC Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test, predictions_bagging):.4f}")
print(classification_report(y_test, predictions_bagging))

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Calculating accuracy
accuracy = accuracy_score(y_test, predictions_bagging)
print(f"Accuracy on test set: {accuracy:.4f}")

# Generating a classification report
report = classification_report(y_test, predictions_bagging)
print("Classification report:\n", report)

# Generating a confusion matrix
conf_matrix = confusion_matrix(y_test, predictions_bagging)
print("Confusion matrix:\n", conf_matrix)


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from itertools import cycle

# Binarize the output
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))

n_classes = y_test_binarized.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], probabilities[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure()

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multiclass')
plt.legend(loc="lower right")
plt.show()




# Plot all ROC curves
Train Your Bagging Classifier with the best-found parameters and ensure it's capable of outputting probabilities (which it should be since you're using SVC with probability=True as the base estimator).

Make Probability Predictions on your test set using the predict_proba method of your trained bagging classifier. This gives you the probability estimates required for ROC curve analysis.

Prepare Your Test Labels for ROC analysis by binarizing them. This step is crucial for multiclass ROC curve plotting, where you're treating each class as a binary classification problem (the class vs. all others).

Compute and Plot ROC Curves for each class using the probability estimates. This gives you insight into how well your model can distinguish between each class and the rest.

Calculate and Plot the Macro-average ROC Curve to get a single summary metric of your model's overall performance across all classes. This macro-average curve helps in understanding the model's general ability to classify instances correctly across a balanced set of classes.

The probability predictions (probabilities = bagging_clf.predict_proba(X_test_normalized)) generated in step 2 are indeed based on your bagging model predictions. This entire process allows you to evaluate how well your ensemble model, specifically the bagging classifier using an SVC base estimator, performs in terms of distinguishing between the different classes in a multiclass classification setting.


In [ ]:
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at these points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot the macro-average ROC curve
plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Macro-average ROC curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

def plot_misclassified_images(indices, X_data, y_true, y_pred, cols=5, rows=5):
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 10))
    axes = axes.flatten()

    for i, ax in zip(indices[:cols*rows], axes):
        image = X_data[i].reshape(28, 28)  # Adjust depending on your data's shape
        ax.imshow(image, cmap='gray')
        ax.set_title(f'True: {y_true[i]}\nPred: {y_pred[i]}')
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Now call the function using the misclassified indices
plot_misclassified_images(misclassified_indices, X_test, y_test, predictions_bagging)


In [ ]:
misclassified_indices = np.where((svc_predictions != y_test) & ((y_test == 9) | (y_test == 7)))[0]

Look for Patterns
As you review the misclassified images, consider questions like:

Are the misclassified 9s written in a particular style that might resemble 7s (or vice versa)?
Is there a consistent reason why these digits are confused, such as similar strokes, angles, or intersections in their shapes?
Are there preprocessing steps that might have distorted these digits or made them less recognizable?
4. Addressing Common Issues
Depending on what you find, you might consider:

Enhancing Data Preprocessing: Adjust your preprocessing steps to better preserve the distinguishing features of 9s and 7s.
Data Augmentation: Introduce variations in your training data that mimic the issues you've identified in the misclassified instances.
Feature Engineering: If using traditional machine learning methods, consider engineering features that better capture the distinct aspects of 9s and 7s.
Model Adjustments: Experiment with different architectures or models that might be better at capturing the nuances between these digits.
5. Iterate
Error analysis is an iterative process. After making adjustments based on your findings, retrain your model and see if the performance on 9s and 7s improves. Repeat the error analysis as necessary to continue refining your model's ability to correctly classify these challenging instances.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming `X_test_normalized` and `y_test` are your test data and labels
predictions = svc_best_model.predict(X_test_normalized)
print(f"Accuracy: {accuracy_score(y_test, predictions):.4f}")
print(classification_report(y_test, predictions))


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming your model can provide probability estimates
# For binary classification or one-vs-rest in multiclass
probabilities = svc_best_model.predict_proba(X_test_normalized)[:, 1]  # Probabilities for the positive class

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, probabilities, pos_label=positive_class_label)  # Define positive_class_label appropriately
roc_auc = auc(fpr, tpr)

# Plotting
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

def evaluate_model(model, X_train, y_train, X_test, y_test):
    model_name = model.__class__.__name__
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Make predictions
    predictions = model.predict(X_test)
    
    # Print metrics
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy_score(y_test, predictions):.4f}")
    print(classification_report(y_test, predictions))
    # Predict on the validation data
    conf_matrix = confusion_matrix(y_test ,predictions)
    ConfusionMatrixDisplay(conf_matrix).plot()
    #dump(model, 'model_',model,'_.joblib')
    

In [ ]:
from sklearn.model_selection import cross_val_score

def evaluate_model_with_cv(model, X_train, y_train, cv=5):
    model_name = model.__class__.__name__
    
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv)
    
    # Print cross-validation metrics
    print(f"Model: {model_name}")
    print(f"Cross-Validation Accuracy Scores: {cv_scores}")
    print(f"Average CV Accuracy: {np.mean(cv_scores):.4f}")
    print(f"Standard Deviation in CV: {np.std(cv_scores):.4f}\n")


In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

   
def plot_learning_curve(model, X, y):
    train_sizes=np.linspace(.1, 1.0, 3)  # Using fewer points
    
    # Start with a subset if needed (e.g., X[:5000], y[:5000]) for large datasets
    # Adjust n_jobs to parallelize computation if your setup supports it
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, train_sizes=train_sizes, cv=5, n_jobs=-1)
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
    plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")
    
    plt.plot(train_sizes, train_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_mean, 'o-', color="g", label="Cross-validation score")
    
    plt.title("Learning Curve")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.show()

# Call the function with a specific model and your data
# plot_learning_curve(model, X_train_normalized, y_train)

